# Storing in Chroma Vector Store DB

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

## 1. RAG 3 Stages: Document Loading, Splitting, Embedding

In [2]:
text_doc = TextLoader(r"..\01_data_ingestion\speech.txt").load()
splitted_docs = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=20).split_documents(text_doc)
ollama_embedder = OllamaEmbeddings(model = 'llama3.2:1b')

C:\Users\RAMU\AppData\Local\Temp\ipykernel_9276\4111092757.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embedder = OllamaEmbeddings(model = 'llama3.2:1b')


## 2. Creating Chroma Vector Store DB:

In [3]:
chroma_db = Chroma.from_documents(splitted_docs, ollama_embedder)

input_text = "We have borne with their present government through all these bitter"
most_matched_docs = chroma_db.similarity_search(input_text)
print(most_matched_docs[0].page_content)

We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.


In [4]:
# We can search the query with score. It will desplay the score also.
docs_and_score=chroma_db.similarity_search_with_score(input_text)
docs_and_score

[(Document(metadata={'source': '..\\01_data_ingestion\\speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there

## 3. Similarity Search with Vector

In [5]:
embedding_vector=ollama_embedder.embed_query(input_text)

searched_docs=chroma_db.similarity_search_by_vector(embedding_vector)

searched_docs

[Document(metadata={'source': '..\\01_data_ingestion\\speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there 

## 4. As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever=chroma_db.as_retriever()
docs=retriever.invoke(input_text)
docs[0].page_content

'We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

## 5. Saving And Loading the DB

In [7]:
### Saving to disk
vectordb=Chroma.from_documents(documents=splitted_docs,embedding=ollama_embedder,persist_directory="./chroma_db")

# Loading from disk
chroma_new_db = Chroma(persist_directory="./chroma_db", embedding_function=ollama_embedder)
docs=chroma_new_db.similarity_search(input_text)

C:\Users\RAMU\AppData\Local\Temp\ipykernel_9276\3119064255.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_new_db = Chroma(persist_directory="./chroma_db", embedding_function=ollama_embedder)
